# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [7]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math


## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('../sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [5]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 56135299, 'calculationPrice': 'close', 'change': 1.35, 'changePercent': 0.00774, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 175.81, 'iexCloseTime': 1692647999480, 'iexLastUpdated': 1692647999480, 'iexMarketPercent': 14.058450087565674, 'iexOpen': 175.1, 'iexOpenTime': 1692624600825, 'iexRealtimePrice': 175.81, 'iexRealtimeSize': 4, 'iexVolume': 706409, 'lastTradeTime': 1692647999904, 'latestPrice': 175.84, 'latestSource': 'Close', 'latestTime': 'August 21, 2023', 'latestUpdate': 1692648000039, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2749123354880, 'oddLotDelayedPrice': None, 'od

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [6]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

29.55

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [15]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [13]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    # print (data)

    for symbol in symbol_string.split(','):
        # print(symbol)
        final_dataframe = pd.concat(
        [
            final_dataframe,
            pd.Series(
                [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
                ],
                index = my_columns
            ).to_frame().T
        ],
        ignore_index = True
        )

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,118.86,26.18,N/A
1,AAL,15.16,3.53,N/A
2,AAP,68.2,10.13,N/A
3,AAPL,175.84,29.55,N/A
4,ABBV,150.16,30.83,N/A
...,...,...,...,...
95,CINF,103.82,12.11,N/A
96,CL,73.8,41.23,N/A
97,CLX,150.07,125.06,N/A
98,CMA,47.33,4.96,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [14]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis = 1, inplace=True)
# len(final_dataframe)

50

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [48]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [16]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [17]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])

final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,CLX,150.07,125.06,133
1,AMZN,134.66,106.03,148
2,CAH,86.94,87.27,230
3,BSX,50.42,84.98,396
4,AMT,175.02,84.55,114
5,ALGN,342.59,84.38,58
6,CDNS,225,69.23,88
7,ABMD,381.02,65.69,52
8,ALXN,182.5,59.45,109
9,ADSK,202.02,52.34,99


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [12]:
symbol = 'AAPL'
batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# print (data[symbol]['advanced-stats'][''])




pe_ratio = data[symbol]['quote']['peRatio']

pb_ratio = data[symbol]['advanced-stats']['priceToBook']

ps_ratio = data[symbol]['advanced-stats']['priceToSales']


enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']

ev_to_ebitda = enterprise_value/ebitda

gross_profit = data[symbol]['advanced-stats']['grossProfit']

ev_to_gross_profit = enterprise_value/gross_profit

16.890783482159986


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [54]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'

]

rv_dataframe = pd.DataFrame(columns=rv_columns)


for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    # print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN


        try:
            ev_to_gross_profit = enterprise_value/ebitda
        except TypeError:
            ev_to_gross_profit = np.NaN

        rv_dataframe = pd.concat(
        [rv_dataframe, 
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
            ],
            index = rv_columns
        ).to_frame().T

        ],
        ignore_index = True
        )

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,117.64,N/A,25.91,N/A,6.01,N/A,4.94,N/A,18.317866,N/A,18.317866,N/A,N/A
1,AAL,14.83,N/A,3.46,N/A,-2.21,N/A,0.1832,N/A,4.665274,N/A,4.665274,N/A,N/A
2,AAP,67.34,N/A,10.01,N/A,1.52,N/A,0.3575,N/A,6.366878,N/A,6.366878,N/A,N/A
3,AAPL,177.23,N/A,29.79,N/A,45.97,N/A,7.22,N/A,22.73089,N/A,22.73089,N/A,N/A
4,ABBV,148.24,N/A,30.44,N/A,20.34,N/A,4.67,N/A,11.369052,N/A,11.369052,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,103.22,N/A,12.04,N/A,1.47,N/A,1.73,N/A,9.055197,N/A,9.055197,N/A,N/A
96,CL,73.51,N/A,41.07,N/A,-949.53,N/A,3.25,N/A,17.95315,N/A,17.95315,N/A,N/A
97,CLX,150.4,N/A,125.33,N/A,84.65,N/A,2.52,N/A,24.251376,N/A,24.251376,N/A,N/A
98,CMA,45.38,N/A,4.75,N/A,1.15,N/A,1.25,N/A,3.409361,N/A,3.409361,N/A,N/A


In [13]:
try:
    ev_to_ebitda = enterprise_value/ebitda
except TypeError:
    ev_to_ebitda = np.NaN


{'week52change': 0.290414, 'week52high': 169.51, 'week52low': 90.41, 'marketcap': 75861250162, 'employees': 10724, 'day200MovingAvg': 134.17, 'day50MovingAvg': 148.64, 'float': 479955020, 'avg10Volume': 1641586.3, 'avg30Volume': 1479579.2, 'ttmEPS': 3.499, 'ttmDividendRate': 0.77, 'companyName': 'Zoetis, Inc.', 'sharesOutstanding': 494883276, 'maxChangePercent': 3.3525, 'year5ChangePercent': 1.69, 'year2ChangePercent': 0.608, 'year1ChangePercent': 0.281583, 'ytdChangePercent': 0.189876, 'month6ChangePercent': 0.10241, 'month3ChangePercent': 0.297048, 'month1ChangePercent': 0.159224, 'day30ChangePercent': 0.165634, 'day5ChangePercent': -0.012817, 'nextDividendDate': None, 'dividendYield': 0.00489706569361443, 'nextEarningsDate': '2020-10-13', 'exDividendDate': '2020-07-01', 'peRatio': 49.1, 'beta': 0.9584403132945124, 'totalCash': 1974321215, 'currentDebt': 549171552, 'revenue': 6518431250, 'grossProfit': 4133660573, 'totalRevenue': 6300683466, 'EBITDA': 2502772553, 'revenuePerShare': 1

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [55]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

In [56]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
6,ABMD,381.02,N/A,65.69,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
28,ALXN,182.5,N/A,59.45,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,323.46,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,94.92,N/A,47.94,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [57]:
rvd_col = [ 'Price',  'Price-to-Earnings Ratio', 'Price-to-Book Ratio', 
       'Price-to-Sales Ratio',  'EV/EBITDA', 'EV/GP']
for column in rvd_col:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [58]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [69]:
from scipy.stats import percentileofscore as score
metrics = {'Price-to-Earnings Ratio': 'PE Percentile', 
            'Price-to-Book Ratio': 'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA': 'EV/EBITDA Percentile', 
            'EV/GP': 'EV/GP Percentile'}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.83,775.067625,3.46,0.126437,-2.2100,0.114943,0.1832,0.034483,4.665274,0.103448,4.665274,0.103448,8.4
1,C,41.02,280.210943,6.53,0.183908,0.4237,0.126437,0.6028,0.137931,3.594677,0.057471,3.594677,0.057471,9.8
2,BA,230.07,49.959807,-30.88,0.045977,-8.9200,0.045977,1.8800,0.454023,-117.864502,0.011494,-117.864502,0.011494,9.9
3,CFG,26.75,429.691696,6.07,0.16092,0.5857,0.137931,1.1300,0.235632,3.732312,0.068966,3.732312,0.068966,11.7
4,AIG,57.34,200.457846,9.60,0.218391,0.9726,0.172414,0.8101,0.16092,3.865561,0.08046,3.865561,0.08046,12.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,ABT,105.15,109.312914,35.77,0.896552,4.9100,0.735632,4.5400,0.793103,23.577194,0.954023,23.577194,0.954023,77.8
83,AJG,222.12,51.747942,43.30,0.931034,4.6000,0.724138,5.2200,0.862069,22.698180,0.931034,22.698180,0.931034,78.4
84,AMZN,134.22,85.637408,105.69,1.0,8.2200,0.885057,2.5700,0.563218,21.867577,0.91954,21.867577,0.91954,78.8
85,ATVI,91.66,125.40097,33.58,0.862069,3.4700,0.62069,8.2800,0.977011,23.865118,0.965517,23.865118,0.965517,79.6


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [70]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.83,775.067625,3.46,0.126437,-2.2100,0.114943,0.1832,0.034483,4.665274,0.103448,4.665274,0.103448,0.096552
1,C,41.02,280.210943,6.53,0.183908,0.4237,0.126437,0.6028,0.137931,3.594677,0.057471,3.594677,0.057471,0.112644
2,BA,230.07,49.959807,-30.88,0.045977,-8.9200,0.045977,1.8800,0.454023,-117.864502,0.011494,-117.864502,0.011494,0.113793
3,CFG,26.75,429.691696,6.07,0.16092,0.5857,0.137931,1.1300,0.235632,3.732312,0.068966,3.732312,0.068966,0.134483
4,AIG,57.34,200.457846,9.60,0.218391,0.9726,0.172414,0.8101,0.16092,3.865561,0.08046,3.865561,0.08046,0.142529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,ABT,105.15,109.312914,35.77,0.896552,4.9100,0.735632,4.5400,0.793103,23.577194,0.954023,23.577194,0.954023,0.866667
83,AJG,222.12,51.747942,43.30,0.931034,4.6000,0.724138,5.2200,0.862069,22.698180,0.931034,22.698180,0.931034,0.875862
84,AMZN,134.22,85.637408,105.69,1.0,8.2200,0.885057,2.5700,0.563218,21.867577,0.91954,21.867577,0.91954,0.857471
85,ATVI,91.66,125.40097,33.58,0.862069,3.4700,0.62069,8.2800,0.977011,23.865118,0.965517,23.865118,0.965517,0.878161


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [71]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace = True)
rv_dataframe = rv_dataframe.loc[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

/tmp/ipykernel_902/846574037.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values('RV Score', ascending=True, inplace = True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.83,775.067625,3.460000,0.126437,-2.210000,0.114943,0.183200,0.034483,4.665274,0.103448,4.665274,0.103448,0.096552
1,C,41.02,280.210943,6.530000,0.183908,0.423700,0.126437,0.602800,0.137931,3.594677,0.057471,3.594677,0.057471,0.112644
2,BA,230.07,49.959807,-30.880000,0.045977,-8.920000,0.045977,1.880000,0.454023,-117.864502,0.011494,-117.864502,0.011494,0.113793
3,CFG,26.75,429.691696,6.070000,0.16092,0.585700,0.137931,1.130000,0.235632,3.732312,0.068966,3.732312,0.068966,0.134483
4,AIG,57.34,200.457846,9.600000,0.218391,0.972600,0.172414,0.810100,0.16092,3.865561,0.08046,3.865561,0.08046,0.142529
5,CMA,45.38,253.288957,4.750000,0.137931,1.150000,0.206897,1.250000,0.298851,3.409361,0.034483,3.409361,0.034483,0.142529
6,AAP,67.34,170.689826,10.010000,0.241379,1.520000,0.275862,0.357500,0.045977,6.366878,0.149425,6.366878,0.149425,0.172414
7,BWA,39.44,291.436432,11.470000,0.275862,1.230000,0.218391,0.549600,0.126437,5.316095,0.126437,5.316095,0.126437,0.174713
8,ALL,105.12,109.34411,-10.130000,0.068966,2.030000,0.390805,0.503400,0.091954,7.055846,0.172414,7.055846,0.172414,0.17931
9,BAC,28.44,404.157977,8.200000,0.195402,0.886500,0.149425,1.520000,0.350575,6.397226,0.16092,6.397226,0.16092,0.203448


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [49]:
portfolio_input()

In [72]:
print (portfolio_size)
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = position_size/rv_dataframe.loc[row, 'Price']

rv_dataframe


1000000


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.83,1296.747757,3.460000,0.126437,-2.210000,0.114943,0.183200,0.034483,4.665274,0.103448,4.665274,0.103448,0.096552
1,C,41.02,468.814462,6.530000,0.183908,0.423700,0.126437,0.602800,0.137931,3.594677,0.057471,3.594677,0.057471,0.112644
2,BA,230.07,83.586601,-30.880000,0.045977,-8.920000,0.045977,1.880000,0.454023,-117.864502,0.011494,-117.864502,0.011494,0.113793
3,CFG,26.75,718.907261,6.070000,0.16092,0.585700,0.137931,1.130000,0.235632,3.732312,0.068966,3.732312,0.068966,0.134483
4,AIG,57.34,335.381396,9.600000,0.218391,0.972600,0.172414,0.810100,0.16092,3.865561,0.08046,3.865561,0.08046,0.142529
5,CMA,45.38,423.771909,4.750000,0.137931,1.150000,0.206897,1.250000,0.298851,3.409361,0.034483,3.409361,0.034483,0.142529
6,AAP,67.34,285.577209,10.010000,0.241379,1.520000,0.275862,0.357500,0.045977,6.366878,0.149425,6.366878,0.149425,0.172414
7,BWA,39.44,487.595569,11.470000,0.275862,1.230000,0.218391,0.549600,0.126437,5.316095,0.126437,5.316095,0.126437,0.174713
8,ALL,105.12,182.941108,-10.130000,0.068966,2.030000,0.390805,0.503400,0.091954,7.055846,0.172414,7.055846,0.172414,0.17931
9,BAC,28.44,676.187385,8.200000,0.195402,0.886500,0.149425,1.520000,0.350575,6.397226,0.16092,6.397226,0.16092,0.203448


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [73]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name= 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [75]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [77]:
column_formats = {
            'A': ['Ticker', string_template ],
            'B': ['Price', dollar_template],
            'C': ['Number of Shares to Buy',integer_template ],
            'D': ['Price-to-Earnings Ratio',float_template],
            'E': ['PE Percentile', percent_template],
            'F': ['Price-to-Book Ratio', float_template],
            'G': ['PB Percentile', percent_template],
            'H': ['Price-to-Sales Ratio', float_template],
            'I': ['PS Percentile', percent_template],
            'J': ['EV/EBITDA',float_template],
            'K': ['EV/EBITDA Percentile', percent_template],
            'L': ['EV/GP', float_template],
            'M': ['EV/GP Percentile', percent_template],
            'N': ['RV Score', percent_template]
}


for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [78]:
writer.close()